<a href="https://colab.research.google.com/github/doeungim/ADP_DE2/blob/main/Correlation_%E2%98%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau, pointbiserialr
import scipy.stats as stats
#!pip install pingouin
import pingouin as pg

In [8]:
# 예제 데이터 생성
np.random.seed(0)
size = 100

df = pd.DataFrame({
    'height': np.random.normal(170, 10, size),                         # 연속형
    'weight': np.random.normal(65, 12, size),                          # 연속형
    'score_rank': np.argsort(np.random.normal(75, 15, size)),         # 순위형
    'binary_sex': np.random.choice([0, 1], size=size),                # 이진형
    'category_major': np.random.choice(['A', 'B', 'C'], size=size),    # 명목형
})
# 순서형 변환
df['score_rank'] = df['score_rank'].rank()

# 이진형 (binary1, binary2) 추가 - Tetrachoric 용
df['binary1'] = np.random.choice([0, 1], size=size)
df['binary2'] = np.random.choice([0, 1], size=size)

# 순서형 추가 (ordinal1, ordinal2) - Polychoric 용
df['ordinal1'] = np.random.choice([1, 2, 3, 4], size=size)
df['ordinal2'] = np.random.choice([1, 2, 3, 4], size=size)

# 다변수 그룹을 위한 컬럼 추가 (Canonical Corr 용)
df['x1'] = np.random.normal(50, 5, size)
df['x2'] = np.random.normal(30, 7, size)
df['y1'] = np.random.normal(20, 10, size)
df['y2'] = np.random.normal(80, 15, size)
df.head(4)

,height,weight,score_rank,binary_sex,category_major,binary1,binary2,ordinal1,ordinal2,x1,x2,y1,y2
0,187.640523,87.597808,72.0,0,B,0,1,4,3,50.073797,37.929189,17.235473,87.841266
1,174.001572,48.826891,38.0,0,C,0,1,4,3,48.044747,33.494163,-1.420405,83.810845
2,179.787380,49.754180,79.0,1,C,0,0,1,1,48.057902,36.021134,31.715163,47.344759
3,192.408932,76.632760,65.0,1,B,0,0,1,2,55.250710,29.700797,20.734699,85.505533


연속형 vs 연속형           → Pearson, Spearman,Kendall
순위형 포함                → Spearman, endall                   
범주형 vs 범주형           → Cramér's V, Chi2기반계수           
연속형 vs 이진형           → Point-Biserial                     
제어 변수 포함             → Partial correlation                
고차원 집합 vs 집합        → Canonical correlation              
비선형 관계까지 포함       → Distance correlation        

In [10]:
# 1. Pearson correlation: height vs weight
r1, p1 = stats.pearsonr(df['height'], df['weight'])
print(p1)

0.268417510217156


In [17]:
# 2. Spearman correlation: score_rank vs weight
results= []
r2, p2 = stats.spearmanr(df['score_rank'], df['weight'])
results.append({
    '분석': 'Spearman',
    '변수': 'score_rank vs weight',
    '귀무가설': '점수 순위와 몸무게 사이에 단조 상관관계가 없다',
    '상관계수': r2,
    'p-value': p2
})
display(pd.DataFrame(results))

,분석,변수,귀무가설,상관계수,p-value
0,Spearman,score_rank vs weight,점수 순위와 몸무게 사이에 단조 상관관계가 없다,-0.070651,0.484864


In [7]:
# Kendall`s tau(켄달의 타우) - 순위간 일치 정도를 측정
# 예) 5명 심사위원의 순위 비교
kendall_r, kendall_p = kendalltau(df['score_rank'], df['weight'])

In [23]:
# Partial Correlation - 제3의 변수를 통제한 상태에서의 두 변수의 관계
results = []
partial = pg.partial_corr(data = df,
                            x = 'height',
                            y = 'weight',
                            covar = 'x1',
                            method = 'pearson')

r5 = partial['r'].values[0]
p5 = partial['p-val'].values[0]
results.append({
    '분석': 'Partial',
    '변수': 'height vs weight (x1 통제)',
    '귀무가설': 'x1을 통제했을 때, height와 weight는 상관관계가 없다',
    '상관계수': r5,
    'p-value': p5
})
display(pd.DataFrame(results))

,분석,변수,귀무가설,상관계수,p-value
0,Partial,height vs weight (x1 통제),"x1을 통제했을 때, height와 weight는 상관관계가 없다",0.11281,0.26625


In [18]:
# Point-Biserial - 하나는 이진변수, 하나는 연속형일떄
# 여성,남성에 따른 시험점수
results = []
r3, p3 = stats.pointbiserialr(df['binary_sex'] , df['weight'])
results.append({
    '분석': 'Point-Biserial',
    '변수': 'binary_sex vs height',
    '귀무가설': '성별과 키 사이에 상관관계가 없다',
    '상관계수': r3,
    'p-value': p3
})
display(pd.DataFrame(results))

,분석,변수,귀무가설,상관계수,p-value
0,Point-Biserial,binary_sex vs height,성별과 키 사이에 상관관계가 없다,-0.102046,0.312368


In [28]:
# Cremer`s V - 범주 2개
results = []
tab = pd.crosstab(df['binary_sex'] , df['category_major'])
chi2 = stats.chi2_contingency(tab)[0]
print(chi2)

n = tab.to_numpy().sum()
phi2 = chi2 / n

r, k = tab.shape

cremer_v = np.sqrt(phi2 / min(k-1, r-1))
results.append({
    '분석': "Cramér's V",
    '변수': 'category_major vs binary_sex',
    '귀무가설': '명목형 변수(category_major, binary_sex) 간의 연관성이 없다',
    '상관계수': cremer_v,
    'p-value': None
})
display(pd.DataFrame(results))

1.6544117647058822


,분석,변수,귀무가설,상관계수,p-value
0,Cramér's V,category_major vs binary_sex,"명목형 변수(category_major, binary_sex) 간의 연관성이 없다",0.128624,None


##### Tetrachoric/Polychoric - 이론적 연속변수의 이진화 OR 등간, 서열 변수의 상관

##### 1. Tetrachoric 상관계수
- 이진형 변수 2개가 사실은 연속형 변수를 일정 기준으로 이진화한 결과라고 가정할때,
-잠재연속 변수들 간의 상관관계를 추정

- binary1, binary 2는 이진이 아닌 정규분포 기반의 연속성이 존재한다
- 귀무) 이진 변수는 서로 독립이며, 이진화 이전의 잠재 연속형 변수들 사이에 상관이 없다.

```python
#!pip install polychoric-python
from polychoric import polychoric_correlation

# 예시 binary data (0과 1로 구성)
binary_df = df[['binary1', 'binary2']]
corr_matrix, _ = polychoric_correlation(binary_df)

print(f'Tetrachoric 상관계수: {corr_matrix[0,1]:.4f}')
```

##### 2. Polychoric 상관계수
- 서열 변수 2개가 사실은 연속형 잠재변수를 기준으로 등간, 순서로 나뉜것이다.
- Ordinal 1 ~ 4 는 실제로는 연속형 변수에서 나온 것이다.
- 귀무) 서열형 변수는 서로 독립이며, 연속 분포 기반 잠배 변수들 간의 상관기 없다.

In [31]:
# Canonical Correlation: [x1,x2] vs [y1,y2]
# 두 변수 집합 간 선형 상관관계가 없다. (귀무)
from sklearn.cross_decomposition import CCA

results = []
cca = CCA(n_components= 1)
cca.fit(df[['x1','x2']] , df[['y1','y2']])

x_c, y_c = cca.transform(df[['x1','x2']] , df[['y1','y2']])
corr = np.corrcoef(x_c[:,0] , y_c[:, 0])[0,1]
print(corr)
results.append({
    '분석': 'Canonical Correlation',
    '변수': '[x1,x2] vs [y1,y2]',
    '귀무가설': '두 변수 집합 간 선형 상관관계가 없다',
    '상관계수': corr,
    'p-value': None
})
display(pd.DataFrame(results))

0.10540967141985796


,분석,변수,귀무가설,상관계수,p-value
0,Canonical Correlation,"[x1,x2] vs [y1,y2]",두 변수 집합 간 선형 상관관계가 없다,0.10541,None
